In [3]:
import json
import pandas as pd
import numpy as np
import os

In [4]:
#file_dir = "C:\Users\15122\Documents\GitHub\Movies-ETL"

file_dir = os.path.join(r"C:\Users\15122\Documents\GitHub", "Movies-ETL")

print(file_dir)

C:\Users\15122\Documents\GitHub\Movies-ETL


In [5]:
#TL;DR: use os.path.join when you need the filepath



#f'{file_dir}filename'

#https://stackoverflow.com/questions/647769/why-cant-pythons-raw-string-literals-end-with-a-single-backslash
    
#don't do it with string concatenation, 
#for it's not the right way to do it anyway! use os.path.join

# import os
# os.path.join(r"C:\mypath", "subfolder")
#'C:\\mypath\\subfolder'

#to end a raw string with a slash I suggest you can use this trick:
#print r"c:\test"'\\'
#test\

#SyntaxError: (unicode error) 'unicodeescape' codec can't 
#decode bytes in position 2-3: truncated \UXXXXXXXX escape


#if you need to prefix a string in a 
#variable X with a backslash then you can do this :

#X='dummy'
#bs=r'\ '

In [6]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [7]:
len(wiki_movies_raw)

7311

In [38]:
# First 5 records
#wiki_movies_raw[:5]

In [37]:
# Some records in the middle
#wiki_movies_raw[3600:3605]

In [18]:
kaggle_path = os.path.join(file_dir, "movies_metadata.csv")
#output: 'C:\\Users\\15122\\Documents\\GitHub\\Movies-ETL\\movies_metadata.csv'
kaggle_metadata = pd.read_csv(kaggle_path, low_memory=False)


ratings_path = os.path.join(file_dir, "ratings.csv")
ratings = pd.read_csv(ratings_path, low_memory=False)

#These do not work as is.
#kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
#ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [19]:
#ratings.sample()

In [20]:
#kaggle_metadata.sample()

In [22]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
#wiki_movies_df.head()
#gives 5 rows × 193 columns

In [39]:
#wiki_movies_df.columns.tolist()

In [27]:
#[expression for element in source_list]
#[expression for element in source_list if filter_expression]

#if ('Director' in movie or 'Directed by' in movie) 
#and 'imdb_link' in movie
#Execute portion of the process


wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
len(wiki_movies)


7080

In [28]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
               and 'imdb_link' in movie 
               and 'No. of episodes' not in movie]

In [30]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
7060,https://en.wikipedia.org/wiki/The_Insult_(film),2018.0,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7293,https://en.wikipedia.org/wiki/Capernaum_(film),2018.0,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

#columns that hold alternate title data: 
#Also known as, Arabic, Cantonese, Chinese, French, Hangul, Hebrew, Hepburn, Japanese, Literally,  Mandarin, McCune–Reischauer, Original title, Polish,  Revised Romanization, Romanized, Russian, Simplified, Traditional, Yiddish

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [43]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    return movie

In [44]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [45]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']